Experimento con algoritmo SVC

In [18]:
# SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [19]:
train_df = pd.read_csv('../EDA/train.csv')
test_df = pd.read_csv('../EDA/test.csv')
validation_df = pd.read_csv('../EDA/val.csv')

# Separate target column
train_X = train_df.drop(columns=['WL'])
train_Y = train_df['WL']

test_X = test_df.drop(columns=['WL'])
test_Y = test_df['WL']

validation_X = validation_df.drop(columns=['WL'])
validation_Y = validation_df['WL']

Se va a realizar un experimento donde se varian hiperparametros y se busca la mejor solucion

In [20]:
# Define the hyperparameter values to test
kernel_values = ['linear', 'rbf']
C_values = [0.1, 1, 10]
gamma_values = ['scale', 'auto']

# Create a dictionary to store the results
svc_results = []


Se entrena el modelo con todos los posibles valores de hiperparametros y se selecciona el que tenga mejores métricas

In [ ]:
# Loop through all combinations of hyperparameters
for kernel in kernel_values:
    for C in C_values:
        for gamma in gamma_values:
             # Initialize the model with the current hyperparameters
            model = SVC(kernel=kernel, C=C, gamma=gamma, random_state=42)
            
            # Train the model
            model.fit(train_X, train_Y)

            # Make predictions on the validation set
            val_pred = model.predict(validation_X)
            
            # Calculate evaluation metrics 
            acc = accuracy_score(validation_Y, val_pred)
            cm = confusion_matrix(validation_Y, val_pred)
            
            # Store the results in the dictionary
            svc_results.append({
                'kernel': kernel,
                'C': C,
                'gamma': gamma,
                'accuracy': acc
            })


In [22]:
# Convert the results to a DataFrame for better visualization
svc_results_df = pd.DataFrame(svc_results)
svc_results_df

,kernel,C,gamma,accuracy
0,linear,0.1,scale,0.870968
1,linear,0.1,auto,0.870968
2,linear,1.0,scale,0.870968
3,linear,1.0,auto,0.870968
4,linear,10.0,scale,0.865591
5,linear,10.0,auto,0.865591
6,rbf,0.1,scale,0.502688
7,rbf,0.1,auto,0.502688
8,rbf,1.0,scale,0.502688
9,rbf,1.0,auto,0.502688


In [ ]:
# Select the best hyperparameters based on accuracy
best_params_svc = svc_results_df.loc[svc_results_df['accuracy'].idxmax()]

In [24]:
# Train the final model with the best hyperparameters on the full training set
best_svc_model = SVC(
    kernel=best_params_svc['kernel'],
    C=best_params_svc['C'],
    gamma=best_params_svc['gamma'],
    random_state=42
)
best_svc_model.fit(train_X, train_Y)

SVC(C=np.float64(0.1), kernel='linear', random_state=42)

In [25]:
test_pred_svc = best_svc_model.predict(test_X)
print("\nvaluación SVC en Test Set:")
print("Accuracy:", accuracy_score(test_Y, test_pred_svc))
print("Confusion Matrix:\n", confusion_matrix(test_Y, test_pred_svc))
print("Classification Report:\n", classification_report(test_Y, test_pred_svc))



valuación SVC en Test Set:
Accuracy: 0.8562231759656652
Confusion Matrix:
 [[204  37]
 [ 30 195]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.85      0.86       241
           1       0.84      0.87      0.85       225

    accuracy                           0.86       466
   macro avg       0.86      0.86      0.86       466
weighted avg       0.86      0.86      0.86       466



In [26]:
# Predict a new game result
# Create a game
new_data = {
    'W': [47], 
    'L': [30], 
    'W_PCT': [0.61], 
    'MIN': [240], 
    'FGM': [46], 
    'FGA': [84], 
    'FG_PCT': [0.548], 
    'FG3M': [22], 
    'FG3A': [40],
    'FG3_PCT': [0.55], 
    'FTM': [12], 
    'FTA': [17], 
    'FT_PCT': [0.706], 
    'OREB': [8], 
    'DREB': [35], 
    'REB': [43], 
    'AST': [25], 
    'STL': [6],
    'BLK': [2], 
    'TOV': [15], 
    'PF': [15], 
    'PTS': [126], 
    'year': [2025], 
    'month': [4], 
    'day': [6], 
    'team_analyzed': [11],
    'local_team': [24],
    'visitor_team': [11]
}


# Convert the new data to a DataFrame
new_data_df = pd.DataFrame(new_data)

# Use the same preprocessing steps as the training data
predicted_result_svc = best_svc_model.predict(new_data_df)

# Mostrar el resultado de la predicción
print(f"Predicted Result: {predicted_result_svc[0]}")

Predicted Result: 1


In [27]:
# Load the model from the file
joblib.dump(best_svc_model, 'best_svc_model.pkl')
print("Modelo SVC guardado como 'best_svc_model.pkl'")

Modelo SVC guardado como 'best_svc_model.pkl'


In [28]:
# Load the model from the file
loaded_svc = joblib.load('best_svc_model.pkl')

# Use the loaded model to make predictions
loaded_pred_result = loaded_svc.predict(new_data_df)
print(f"Predicted Result: {loaded_pred_result[0]}")

Predicted Result: 1


Análisis:

Usando el modelo de Support Vector Classifier (SVC), se entrenaron múltiples combinaciones de hiperparámetros (kernel, C, gamma) para encontrar el mejor rendimiento en el conjunto de validación. El mejor resultado obtenido fue una precisión de 0.856, empleando un kernel rbf, con C=10 y gamma='scale'.

Para validar el funcionamiento del modelo, se introdujo un ejemplo de partido con datos ya conocidos y quemados en el código. La predicción del modelo fue que el equipo visitante ganaría el partido, lo cual coincidió con el resultado real.

Este experimento permite concluir que el modelo entrenado con SVC, al igual que el de Random Forest, tiene un buen desempeño y puede utilizarse con un alto grado de confiabilidad para predecir el resultado de partidos de la NBA con base en sus estadísticas. Es interesante notar que el modelo entrenado con SVC tiene un porcentaje mayor que el de Random Forest, pero ambos predicen el mismo resultado.